In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
from tqdm import tqdm
from numpy.polynomial import Polynomial
import warnings
import wrds
warnings.filterwarnings('ignore')

In [10]:
def blackscholes_mc(S=100, vol=0.2, r=0, q=0, ts=np.linspace(0, 1, 13), npaths=10):
    nsteps = len(ts) - 1
    ts = np.asfarray(ts)[:, np.newaxis]
    W = np.cumsum(np.vstack((np.zeros((1, npaths), dtype=float),
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))),
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r-q)*ts)
    return paths


def blackscholes_price(K, T, S, vol, r=0, q=0, callput='call'):
    F = S*np.exp((r-q)*T)
    v = np.sqrt(vol**2*T)
    d1 = np.log(F/K)/v + 0.5*v
    d2 = d1 - v
    try:
        opttype = {'call':1, 'put':-1}[callput.lower()]
    except:
        raise ValueError('The value of callput must be either "call" or "put".')
    price = opttype*(F*norm.cdf(opttype*d1)-K*norm.cdf(opttype*d2))*np.exp(-r*T)
    return price

In [11]:
CONFIG = {
    'X0': 100,
    'T': 1.0,
    'r': 0.0,
    'q': 0.0,
    
    'sigma_low': 0.1,
    'sigma_high': 0.2,
    'sigma_hat': 0.15,
    
    'K1': 90,
    'K2': 110,
    
    'n_steps_1': 12,
    'n_paths_1': 5000,

    'n_steps_2': 360,
    'n_paths_2': 50000,
    
    'poly_deg': 5,
}

In [12]:
def Sigma(gamma):
    return np.where(gamma >= 0, CONFIG['sigma_high'], CONFIG['sigma_low'])


def payoff(x):
    K1, K2 = CONFIG['K1'], CONFIG['K2']
    return 100 / (K2 - K1) * (np.maximum(x - K1, 0) - np.maximum(x - K2, 0))


def payoff_deriv(x):
    K1, K2 = CONFIG['K1'], CONFIG['K2']
    return 100 / (K2 - K1) * ((x > K1).astype(float) - (x > K2).astype(float))


In [13]:
def make_get_Gamma(Gamma_polys, n_steps, T, sigma_hat):
    """Create Gamma interpolation function."""
    dt = T / n_steps
    
    def get_Gamma(t, x):
        idx = int(t * n_steps / T)
        idx = min(idx, n_steps - 1)
        poly = Gamma_polys[idx]
        return poly(x) / (dt * sigma_hat * x)
    
    return get_Gamma

In [14]:
np.random.seed(42)

# Unpack config
X0 = CONFIG['X0']
T = CONFIG['T']
sigma_hat = CONFIG['sigma_hat']
n_steps_1 = CONFIG['n_steps_1']
n_paths_1 = CONFIG['n_paths_1']
deg = CONFIG['poly_deg']

# Generate paths for BSDE
ts_1 = np.linspace(0, T, n_steps_1 + 1)
dt_1 = np.diff(ts_1)

dW_1 = np.random.randn(n_steps_1, n_paths_1) * np.sqrt(dt_1[:, np.newaxis])
W_1 = np.vstack([np.zeros((1, n_paths_1)), np.cumsum(dW_1, axis=0)])

paths_1 = X0 * np.exp(-0.5 * sigma_hat**2 * ts_1[:, np.newaxis] + sigma_hat * W_1)

# Backward recursion
Y = payoff(paths_1[-1])
Z = payoff_deriv(paths_1[-1])

Gamma_polys = [None] * n_steps_1

for i in range(n_steps_1, 0, -1):
    X_prev = paths_1[i-1]
    dt = dt_1[i-1]
    dW = dW_1[i-1]
    
    poly_Gamma = Polynomial.fit(X_prev, Z * dW, deg)
    Gamma = poly_Gamma(X_prev) / (dt * sigma_hat * X_prev)
    Gamma_polys[i-1] = poly_Gamma

    poly_Z = Polynomial.fit(X_prev, Y * dW, deg)
    Z = poly_Z(X_prev) / (dt * sigma_hat * X_prev)
    
    poly_Y = Polynomial.fit(X_prev, Y, deg)
    E_Y = poly_Y(X_prev)
    Y = E_Y + 0.5 * Gamma * X_prev**2 * (Sigma(Gamma)**2 - sigma_hat**2) * dt

step1_price = np.mean(Y)
print(f"Step 1 price: {step1_price:.4f}")
print(f"True price: 56.0")
print(f"Error: {abs(step1_price - 56) / 56 * 100:.2f}%")

Step 1 price: 56.0051
True price: 56.0
Error: 0.01%


In [15]:
np.random.seed(123)

# Unpack config
n_steps_2 = CONFIG['n_steps_2']
n_paths_2 = CONFIG['n_paths_2']

# Create Gamma function
get_Gamma = make_get_Gamma(Gamma_polys, n_steps_1, T, sigma_hat)

# Generate paths with uncertain volatility
dt_2 = T / n_steps_2

paths_2 = np.zeros((n_steps_2 + 1, n_paths_2))
paths_2[0] = X0

for i in range(n_steps_2):
    t = i * dt_2 
    X = paths_2[i]
    
    Gamma = get_Gamma(t, X)
    sigma = Sigma(Gamma)
    
    dW = np.random.randn(n_paths_2) * np.sqrt(dt_2)
    paths_2[i+1] = X * np.exp(-0.5 * sigma**2 * dt_2 + sigma * dW)

# Price the call spread
step2_price = np.mean(payoff(paths_2[-1]))
print(f"Step 2 price: {step2_price:.4f}")
print(f"True price: 56.0")
print(f"Error: {abs(step2_price - 56) / 56 * 100:.2f}%")

Step 2 price: 54.4959
True price: 56.0
Error: 2.69%


In [16]:
conn = wrds.Connection()

print("Connected to WRDS successfully!")
print(f"Available libraries: {conn.list_libraries()[:5]}...")

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Connected to WRDS successfully!
Available libraries: ['aha_sample', 'ahasamp', 'audit', 'audit_audit_comp', 'audit_common']...


In [17]:
tables = conn.list_tables(library='optionm')
print(f"OptionMetrics tables: {tables[:10]}...")

OptionMetrics tables: ['borrate1996', 'borrate1997', 'borrate1998', 'borrate1999', 'borrate2000', 'borrate2001', 'borrate2002', 'borrate2003', 'borrate2004', 'borrate2005']...


In [18]:
secid_query = """
    SELECT secid, ticker, effect_date
    FROM optionm.securd
    WHERE ticker = 'SPY'
"""
spy_info = conn.raw_sql(secid_query)
print("SPY info:")
print(spy_info)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "effect_date" does not exist
LINE 2:     SELECT secid, ticker, effect_date
                                  ^

[SQL: 
    SELECT secid, ticker, effect_date
    FROM optionm.securd
    WHERE ticker = 'SPY'
]
(Background on this error at: https://sqlalche.me/e/20/f405)